# Модуль сегментации на предложения, токенизации, лемматизации и морфологического анализа

### Использование библиотек проекта *Natasha*

Для обработки нашего корпуса был использован набор Python-библиотек для обработки текстов на естественном русском языке **проекта Natasha**. Мы специально используем не базовую компактную библиотеку *natasha*, a отдельные профессиональные библиотеки проекта, качество и производительность которых выше, да и вообще это было вполне удобно. Итак, вот они: 
  * **Slovnet** — компактные модели для обработки естественного русского языка; нами использовался морфологический теггер.
  * **Razdel** — инструмент для сегментации текста на предложения и токены.
  * **Navec** — набор компактных предобученных эмбеддингов для русского языка.
  * **Natasha** — из неё мы взяли лемматизатор, так как он, к нашему удивлению, не встроен в Slovnet.
К тому же, *Slovnet* и *Navec* были обучены на новостях, а мы работаем с публицистикой, пусть и не совсем новостной.

### База данных

Предложения (в формате *id_sent, id_text, sent*) и токены (в формате *id_token, id_sent, id_text, token, lemma, pos, morph*)  записываются в таблицы *df_sents* и *df_tokens* базы данных *project_nlp.db* соответственно. Важно сказать, что кода для создания таблиц нет, так как они создавались с помощью приложения *DB Browser for SQLite*.

### Для тестирования

Дорогие преподаватели, если вы хотите протестировать следующие ячейки кода, советую вам закомментировать и раскомментировать строки, у которых указан соответствующий комментарий. В базе данных находится более 600 000 тысяч словоупотреблений, она относительно долго собиралась.

### Морфологический анализ и лемматизация

Функция *tokens(chunk, id_sents, id_text)* производит морфологический анализ токена с помощью *Slovnet*, основанный на контексте, находит лемму с помощью *morph_vocab.lemmatize()* из *Natasha* и делает запись в базу данных в формате описанном выше. На вход получает список *chunk*, представляющий собой список предложений, представленных в виде списков токенов (извините за тавтологию), а также id текста и предложения.
Теги частей речи и морфологические теги являются [Universal POS tags](https://universaldependencies.org/u/pos/).

In [38]:
def tokens(chunk, id_sents, id_text):
    x = morph.map(chunk)
    tokens = []
    for id_s in id_sents:
        markup = next(x)
        count_tok = 0
        for token in markup.tokens:
            t = list(token)
            if t[1] != 'PUNCT':
                lemma = morph_vocab.lemmatize(t[0], t[1], t[2])
                t.append(lemma)
                grammemes = t.pop(2)
                t.insert(2, ','.join([':'.join([x, y]) for x, y in list(grammemes.items())]))
                t.append(id_text)
                t.append(id_s)
                t.append(counter_token)
                token_to_db = tuple([t[6], t[5], t[4], t[0], t[3], t[1], t[2]])
                # print(token_to_db)
                c.execute('INSERT INTO df_tokens VALUES (?, ?, ?, ?, ?, ?, ?)', token_to_db) # это можно закомментировать для тестирования
                counter_token += 1
                con.commit() # это можно закомментировать для тестирования
                count_tok += 1
            markup = next(morph.map(chunk))

### Сегментация на предложения и токенизация

Функция *sents_with_tokens(row)* производит сегментацию текста на предложения и токенизацию с помощью инструментов *Razdel*, а затем вписывает информацию о предложении в базу данных в форфмате, описанном выше. После этого вызывает функцию *tokens(chunk, id_sents, id_text)*. На вход получает кортеж из id текста и самого текста.

In [39]:
def sents_with_tokens(row):
    counter = 0
    sents = list(sentenize(row[1]))
    chunk = []
    sents_final = []
    id_sents = []
    for s in sents:
        sent_to_db = []
        id_sent = row[0] + '_' + str(counter)
        id_sents.append(id_sent)
        sent_to_db.append(id_sent)
        sent_to_db.append(row[0])
        sent_to_db.append(list(s)[2])
        sent_to_db = tuple(sent_to_db)
        # print(sent_to_db)
        c.execute('INSERT INTO df_sents VALUES (?, ?, ?)', sent_to_db) # это можно закомментировать для тестирования
        con.commit() # это можно закомментировать для тестирования
        tokens_ch = [_.text for _ in tokenize(s.text)]
        chunk.append(tokens_ch)
        counter += 1
    tokens(chunk, id_sents, row[0])

### Активация функций и заполнение базы данных

Подключаемся к базе данных.

In [40]:
con = sqlite3.connect('project_nlp.db')
c = con.cursor()
con.commit()

Здесь вызывается объект *MorphVocab()* для лемматизации и загружаются модели ([эмбеддинги](https://github.com/natasha/navec) и [морфоанализатор](https://github.com/natasha/slovnet); второй будет приложен на GitHub, первый не получается приложить, так как он весит 25 Мб, но его можно скачать по ссылке) для работы морфолгоческого анализатора. Также выкачивается список пар (*id_text, text*) для последующей обработки. Делать это без такого списка у нас не получилось. Видимо, из-за того, что с такими объёмами информации *sqlite3* работает нестабильно.

In [41]:
from razdel import sentenize
from razdel import tokenize
from navec import Navec
from slovnet import Morph
from natasha import MorphVocab

navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
morph = Morph.load('slovnet_morph_news_v1.tar', batch_size=4)
morph.navec(navec)

morph_vocab = MorphVocab()

rows = []
counter_token = 0
for row in c.execute('SELECT id_text, text FROM df_texts'):
    rows.append(row)

Здесь для каждого текста из базы данных активируется функция *sents_with_tokens(row)* (и, соответственно, *tokens(chunk, id_sents, id_text)*).

In [42]:
for r in tqdm(rows):
    sents_with_tokens(r)
    # break

100%|██████████| 259/259 [1:02:27<00:00, 14.47s/it]


Закрываем базу данных.

In [46]:
con.close()